In [ ]:
import numpy as np
import pandas as pd
import param
import panel as pn
pn.extension()
import hvplot.pandas

In [ ]:
df_cap = pd.read_csv('../results/capacity.csv')
df_fuel = pd.read_csv('../results/fuel_cons.csv')
df_ELsup = pd.read_csv('../results/Elsup.csv')
df_emissions = pd.read_csv('../results/emissions.csv')
df_trade = pd.read_csv('../results/trade.csv')
df_investments = pd.read_csv('../results/investments.csv')

In [ ]:
df_investments['value'] = np.divide(df_investments['value'],1000)

In [ ]:
names = {'bah':'Bahrain',
         'kuw':'Kuwait',
         'ksa':'Saudi Arabia',
         'omn':'Oman',
         'qat':'Qatar',
         'uae':'UAE'}

df_cap = df_cap.replace(names)
df_fuel = df_fuel.replace(names)
df_ELsup = df_ELsup.replace(names)
df_emissions = df_emissions.replace(names)
df_trade = df_trade.replace(names)
df_investments = df_investments.replace(names)

In [ ]:
countries = ['Bahrain','Kuwait','Oman','Qatar','Saudi Arabia','UAE']
fuels = ['HFO','diesel','NG','Arablight']
scenarios = ['A','B','C','D','E','F','G','H']

In [ ]:
df_fuel.head(2)

In [ ]:
class GENERAL(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    scenario_selector = param.ObjectSelector(default="H", objects=scenarios)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")

# CAPACITY ----------------------------------------------------------------   

    def get_data_cap(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        #country = self.country_selector
        
        mask =  (df_cap['c'] == country) & (df_cap['trun'] >= start_date) & (df_cap['trun'] <= end_date)
        
        _df = df_cap.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def plot_cap(self):
        country = self.country_selector
        _df  = self.get_data_cap(country)
        mask = (_df['scenario']==self.scenario_selector)
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = country + ' ' + self.scenario_selector
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=ELp_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               legend=self.show_legend,
                               shared_axes=False,
                               rot=90)
        return plot

# FUEL ----------------------------------------------------------------    
    
    def get_data_fuel(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        #country = self.country_selector
        
        mask = (df_fuel['c'] == country) & (df_fuel['trun'] >= start_date) & (df_fuel['trun'] <= end_date)
        
        _df = df_fuel.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df

    def plot_fuel(self):     
        country = self.country_selector
        _df  = self.get_data_fuel(country)
        mask = (_df['scenario']==self.scenario_selector)               # come back to this
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector + ' ' + self.scenario_selector
        
        fuel_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=fuel_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               legend=self.show_legend,
                               shared_axes=False,
                               rot=90)
        return plot
    

# ELECTRICITY SUPPLY ---------------------------------------------------------------   

    def get_data_sup(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        #country = self.country_selector
        
        mask =  (df_ELsup['c'] == country) & (df_ELsup['trun'] >= start_date) & (df_ELsup['trun'] <= end_date)
        
        _df = df_ELsup.loc[mask]
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='ELp',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        return df_pivot
    
    def plot_sup(self):
        country = self.country_selector
        _df  = self.get_data_sup(country)
        mask = (_df['scenario']==self.scenario_selector)
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector + ' ' + self.scenario_selector
        
        ELp_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=ELp_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               legend=self.show_legend,
                               shared_axes=False,
                               rot=90)
        return plot

# EMISSIONS ---------------------------------------------------------------   

    def get_data_emissions(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        
        mask = (df_emissions['c'] == country) & (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
        _df = df_emissions.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_emissions(self):
        country = self.country_selector
        _df  = self.get_data_emissions(country)
        mask = (_df['scenario']==self.scenario_selector)
        data = _df.loc[mask].reset_index(drop=True)   
        
        title = 'Emissions for ' + country + ' for ' + self.scenario_selector
        
        #c_list = data.drop(['trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot(
            x='trun',
            value_label='million tons',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=False,
            rot=90)* data.hvplot.scatter(
            x='trun',
            title=title,
            line_color='k',
            shared_axes=True)
        return plot

    

In [ ]:
class EMISSIONS(GENERAL):
    
# EMISSIONS SUBPLOTS
# https://hvplot.holoviz.org/user_guide/Subplots.html

    def plot_emissions_grid(self):
        country = self.country_selector
        _df  = self.get_data_emissions(country)
        
        title = 'Emissions for ' + country
        
        plot = (_df.hvplot(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            value_label='million tons',
            alpha=0.7,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * _df.hvplot.scatter(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            title=title,
            line_color='k',
            shared_axes=True)).cols(4)
        return plot

# EXCHANGE ---------------------------------------------------------------  
#make own object

# INVESTMENTS ------------------------------------------------------------
#make own object

In [ ]:
class FUEL(GENERAL):
    def plot_fuel_grid(self):    

        country = self.country_selector
        _df  = self.get_data_fuel(country)
        ##mask = (_df['scenario']==self.scenario_selector)               # come back to this
        ##data = _df.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector
        
        fuel_list = _df.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = _df.hvplot.bar(
            x='trun',
            y='value',
            by='scenario',
            #stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot    
    
    def plot_fuel_grid2(self):    

        country = self.country_selector
        _df  = self.get_data_fuel(country)
        ##mask = (_df['scenario']==self.scenario_selector)               # come back to this
        ##data = _df.loc[mask].reset_index(drop=True)      
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        title = self.country_selector
        
        fuel_list = df_pivot.drop(['c','trun','scenario'],axis=1).columns.values.tolist()
        display(df_pivot)

        # workaround for no stacked bar chart by scenario
        # does not stack though
        plot = df_pivot.hvplot.bar(
            x='trun',
            y='NG',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * df_pivot.hvplot.bar(
            x='trun',
            y='Arablight',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * df_pivot.hvplot.bar(
            x='trun',
            y='HFO',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)* df_pivot.hvplot.bar(
            x='trun',
            y='diesel',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot  

In [ ]:
#A = GENERAL()
#A.country_selector = "Kuwait"
#A.scenario_selector = "B"
##A.plot_cap()
##A.plot_fuel()
##A.plot_sup()
#A.plot_emissions()
#
#B = GENERAL()
#B.country_selector = "Oman"
##B.scenario_selector = "C"
#B.plot_emissions()

#C = EMISSIONS()
#C.country_selector = "Qatar"
##C.scenario_selector = "D"
#C.plot_emissions_grid()

D = FUEL()
D.country_selector = "Saudi Arabia"
D.plot_fuel_grid2()

In [ ]:
class ALL(GENERAL):
    def make_gridplot(self):
        
        # create three plots
        s1 = self.plot_cap()
        s2 = self.plot_fuel()
        s3 = self.plot_sup()
        s4 = self.plot_emissions()
    
        
        # make a grid
        grid = gridplot([s1,s2], ncols=2, plot_width=250, plot_height=250)
        return grid

In [ ]:
class INVESTMENTS(param.Parameterized):
    show_legend = param.Boolean(False, doc="Show the legend")
    
    def get_data(self):
        _df = df_investments
        
        _df = (_df
               .drop(['trun'],axis=1)
               .pivot_table(index=['scenario'],
                            columns=['tech'],
                            fill_value=0,
                            aggfunc=np.sum)
              )
        
        _df.columns = _df.columns.droplevel()
        return _df

    def plot_data(self):
        data = self.get_data()
        
        #_df  = self.get_data()
             
        title = 'Cumulative investments'
        ELp_list = data.columns.values.tolist()

        plot = data.hvplot.bar(value_label='billion USD',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=800,
                               height=600)
        return plot
    
inv = INVESTMENTS(name='')

In [ ]:
dashboard = pn.Tabs(('At a glance',
                     pn.Column(
                     pn.Row(g.param),
                     pn.Row(g.plot_cap,g.plot_fuel),
                     pn.Row(g.plot_el,g.plot_emissions))
))

dashboard.append(('Capacity',
             pn.Column(pn.Row('## Capacities'),
             pn.Row(cap.param),
             pn.Row(cap.cap_plot_A,cap.cap_plot_B,cap.cap_plot_C,cap.cap_plot_D),
             pn.Row(cap.cap_plot_E,cap.cap_plot_F,cap.cap_plot_G,cap.cap_plot_H))                      
            ))

dashboard.append(('Fuel',
             pn.Column(pn.Row('## Fuel consumption'),
                       pn.Row(f.param),
                       pn.Row(f.fuel_plot_A,f.fuel_plot_B,f.fuel_plot_C,f.fuel_plot_D),
                       pn.Row(f.fuel_plot_E,f.fuel_plot_F,f.fuel_plot_G,f.fuel_plot_H))          
            ))

dashboard.append(('Electricity',
             pn.Column(
             pn.Row('## Electricity supply'),
             pn.Row(el.param),
             pn.Row(el.plot_A,el.plot_B,el.plot_C,el.plot_D),
             pn.Row(el.plot_E,el.plot_F,el.plot_G,el.plot_H))                      
            ))

dashboard.append(('Emissions',
             pn.Column(
             pn.Row('## CO2 emissions'),
             pn.Row(em.param),
             pn.Row(em.plot_A,em.plot_B,em.plot_C,em.plot_D),
             pn.Row(em.plot_E,em.plot_F,em.plot_G,em.plot_H))                      
            ))

dashboard.append(('Exchange',
             pn.Column(
                 pn.Row('## Electricity exchange'),
                 pn.Row(ex.param),
                 pn.Row(ex.plot_C,ex.table_C),
                 pn.Row(ex.plot_D,ex.table_D),
                 pn.Row(ex.plot_G,ex.table_G),
                 pn.Row(ex.plot_H,ex.table_H))
             ))

dashboard.append(('Investment',
                 pn.Column(
                     pn.Row(inv.param),
                     pn.Row(inv.plot_data))
                 ))

dashboard.append(('Side by side',
                 pn.Column(pn.Row(c.param.country_selector1,c.param.scenario_selector,c.param.show_legend),
                 pn.Row(c.param.country_selector2,c.param.year_range),
                 pn.Row('## Side by side country comparisons'),
                 pn.Row(c.cap_plot1,c.cap_plot2),
                 pn.Row(c.fuel_plot1,c.fuel_plot2),
                 pn.Row(c.sup_plot1,c.sup_plot2))
                 ))

In [ ]:
dashboard.servable()

In [ ]:
#! panel serve --show --port 5000 dashboard.ipynb